In [1]:
import requests

In [2]:
from bedrock.client.client import BedrockAPI

In [3]:
SERVER = "http://localhost:81/"

In [4]:
api = BedrockAPI(SERVER)

In [5]:
filepath =  '/home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv'

In [6]:
ingest_id = 'opals.spreadsheet.Spreadsheet.Spreadsheet'
resp = api.put_source('Rand2001',ingest_id, 'default',{'file':open(filepath, "rb")})
if resp.status_code == 201:
    print 'Source {0} successfully uploaded'.format(filepath)
else:
    print 'Failed to update {0}'.format(filepath)

INFO: 2017-09-05 12:06:12,626: putting source to: http://localhost:81/dataloader/sources/Rand2001/opals.spreadsheet.Spreadsheet.Spreadsheet/default/


Failed to update /home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv


In [7]:
available_sources = api.list("dataloader","sources").json()
source = (s for s in available_sources if s["name"] == "Rand2001").next()
source_id = source["src_id"]
endpoint = api.endpoint("dataloader","sources/%s/explore/" % source_id)
resp = requests.get(endpoint)

INFO: 2017-09-05 12:06:12,810: Listing: http://localhost:81/dataloader/sources/


In [8]:
resp = api.create_matrix(source_id, 'rand_mtx')
mtx = resp[0]
matrix_id = mtx['id']
print(mtx)
#resp

{u'src_id': u'2876d3ecda384e6ca195fefc0de078e9', u'name': u'rand_mtx', u'created': u'2017-09-05 16:06:13.255521', u'outputs': [u'features_original.txt', u'features.txt', u'matrix.csv'], u'id': u'28c95f5637ff4631a8b20d5fd98348a2', u'rootdir': u'/opt/bedrock/dataloader/data/2876d3ecda384e6ca195fefc0de078e9/28c95f5637ff4631a8b20d5fd98348a2/', u'filters': {u'fluid_dummy': {}, u'previous_decision': {}, u'group_size': {}, u'decision0d1c': {}, u'playerid': {}, u'_Icondition_4': {}, u'_Icondition_3': {}, u'_Icondition_2': {}, u'sessionnum': {}, u'num_neighbors': {}, u'round_num': {}, u'condition': {}}, u'mat_type': u'csv'}


In [9]:
resp = api.analytic('opals.stan.Stan.Stan_GLM')
if resp.json():
    print("Stan_GLM Opal Installed!")
else:
    print("Stan_GLM Opal Not Installed!")

Stan_GLM Opal Installed!


In [10]:
resp

<Response [200]>

## Run Stan_GLM

In [11]:
analytic_id = "opals.stan.Stan.Stan_GLM"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
            { "name" : "Formula", "attrname" : "formula", "value" : "decision0d1c ~ condition", "type" : "input" },
            { "name" : "GLM family", "attrname" : "family", "value" : "binomial(link = 'logit')", "type" : "input" },
            { "name" : "chains", "attrname" : "chains" , "value" : "3", "type" : "input"},
            { "name" : "iter", "attrname" : "iter" , "value" : "3000", "type" : "input"},
            { "name" : "prior", "attrname" : "prior" , "value" : "", "type" : "input"},
            { "name" : "prior_intercept", "attrname" : "prior_intercept" , "value" : "", "type" : "input"}
        ]

result_mtx = api.run_analytic(analytic_id, mtx, 'rand_stan_test', input_data=inputData, parameter_data=paramsData)

result_mtx

{u'analytic_id': u'opals.stan.Stan.Stan_GLM',
 u'created': u'2017-09-05 16:07:20.026469',
 u'id': u'4ccd7d9f480648ca83dcb2486f6f36ef',
 u'name': u'rand_stan_test',
 u'outputs': [u'prior_summary.txt', u'summary.txt'],
 u'parameters': [{u'attrname': u'formula',
   u'name': u'Formula',
   u'type': u'input',
   u'value': u'decision0d1c ~ condition'},
  {u'attrname': u'family',
   u'name': u'GLM family',
   u'type': u'input',
   u'value': u"binomial(link = 'logit')"},
  {u'attrname': u'chains',
   u'name': u'chains',
   u'type': u'input',
   u'value': u'3'},
  {u'attrname': u'iter',
   u'name': u'iter',
   u'type': u'input',
   u'value': u'3000'},
  {u'attrname': u'prior', u'name': u'prior', u'type': u'input', u'value': u''},
  {u'attrname': u'prior_intercept',
   u'name': u'prior_intercept',
   u'type': u'input',
   u'value': u''}],
 u'rootdir': u'/opt/bedrock/analytics/data/28c95f5637ff4631a8b20d5fd98348a2/4ccd7d9f480648ca83dcb2486f6f36ef/',
 u'src_id': u'28c95f5637ff4631a8b20d5fd98348a2'

## Get Stan_GLM outputs

In [14]:
result_mtx['src_id']

u'28c95f5637ff4631a8b20d5fd98348a2'

In [15]:
result_mtx['id']

u'4ccd7d9f480648ca83dcb2486f6f36ef'

In [12]:
prior_summary = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'prior_summary.txt')
prior_summary



ERROR: 2017-09-05 12:07:20,043: Unknown remote file format


In [13]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.txt')
summary_table



ERROR: 2017-09-05 12:07:20,052: Unknown remote file format
